<a href="https://colab.research.google.com/github/V-Sekai-fire/meshgpt-pytorch/blob/main/MeshGPT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We use a local runtime. https://research.google.com/colaboratory/local-runtimes.html

In [24]:
#!bash <(curl -L micro.mamba.pm/install.sh) 
#Prefix location? [~/micromamba] /workspace/micromamba
#!pip3 install notebook jupyterlab
#!micromamba create -n py311_meshgpt python=3.11 anaconda -c pytorch -c conda-forge -c anaconda -c defaults -y
#!micromamba activate py311_meshgpt
#!jupyter notebook --port=9999  --NotebookApp.port_retries=0 --allow-root
# Login with kernel
!pip3 install git+https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git
!pip3 install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
!pip3 install trimesh

  Cloning https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git to /tmp/pip-req-build-k11vcwtf
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git /tmp/pip-req-build-k11vcwtf
  Resolved https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git to commit 698c83562f8859c763880f200b210ff1081efedc
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-pxyk2f_w
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-pxyk2f_w
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit d35228edc46b550dc8faeefb832db7bd43a23c2a
  Preparing metadata (setup.py) ... done


In [25]:
from re import T
is_train_autoencoder = True
is_train_autoencoder_disable_iteration = True
is_train_mesh_transformer = True
is_clear_dataset_npz = True


In [26]:
import torch
import trimesh
import numpy as np
import os
import csv
from collections import OrderedDict

from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer
)
from meshgpt_pytorch.data import (
    derive_face_edges_from_faces
)

In [27]:
import trimesh
import torch
import numpy as np
import os
from collections import OrderedDict

max_faces = 4096

def get_mesh(file_path):
    mesh = trimesh.load(file_path, force='mesh')

    # Center and scale vertices
    center = np.mean(mesh.vertices, axis=0)
    vertices = mesh.vertices - center
    max_abs = np.max(np.abs(vertices))
    scale_factor = (1 / 128) / max_abs
    vertices *= scale_factor

    # Quantize vertices
    vertices = np.around(vertices).astype(np.float32)

    # Sort vertices by Z, Y, X
    sorted_indices = np.lexsort(vertices.T[::-1])
    vertices = vertices[sorted_indices]

    # Map old indices to new, sorted indices
    vertex_map = np.empty(len(sorted_indices), dtype=int)
    vertex_map[sorted_indices] = np.arange(len(sorted_indices))

    # Reindex faces
    reindexed_faces = vertex_map[mesh.faces]
    sorted_faces = np.sort(reindexed_faces, axis=1)

    return vertices, sorted_faces

def augment_mesh(vertices, jitter_strength=0.01):
    jitter_amount = np.random.uniform(-jitter_strength, jitter_strength, size=vertices.shape)
    vertices += jitter_amount
    return vertices

def snake_to_sentence_case(snake_str):
    components = snake_str.split("_")
    return " ".join(word.capitalize() for word in components)

def load_filename(directory, variations):
    obj_datas = []

    # Get random scale factors within a range
    scale_factors = np.random.uniform(0.75, 1.0, size=variations)

    for filename in os.listdir(directory):
        if filename.endswith(".glb"):
            file_path = os.path.join(directory, filename)
            vertices, faces = get_mesh(file_path)

            if len(faces) > max_faces:
                print(f"Mesh {filename} has {len(faces)} faces which is more than the allowed {max_faces} faces. Rejecting.")
                continue

            faces_tensor = torch.tensor(faces, dtype=torch.long).to("cuda")
            face_edges = derive_face_edges_from_faces(faces_tensor)

            text, _ = os.path.splitext(filename)
            text = snake_to_sentence_case(text)
            # Run video llava on the image. "Describe the focus of the photo as a json dictionary."
            for scale_factor in scale_factors:
                aug_vertices = augment_mesh(vertices.copy()) * scale_factor
                aug_vertices_tensor = torch.tensor(aug_vertices, dtype=torch.float)

            obj_data = {
                "vertices": aug_vertices_tensor.to("cuda"),
                "faces": faces_tensor,
                "face_edges": face_edges,
                "texts": text
            }
            obj_datas.append(obj_data)

    print(f"[create_mesh_dataset] Returning {len(obj_datas)} meshes")
    return obj_datas


In [28]:
from pathlib import Path
import gc
import torch
import os
from meshgpt_pytorch import MeshDataset

torch.cuda.empty_cache()
gc.collect()

working_dir = f'/workspace/meshgpt-pytorch/dataset/blockmesh_test'

working_dir = Path(working_dir)
working_dir.mkdir(exist_ok = True, parents = True)
dataset_path = working_dir / (project_name + ".npz")


if is_clear_dataset_npz:
    data = load_filename(working_dir, 1)
    dataset = MeshDataset(data)
    dataset.generate_face_edges()
    dataset.save(dataset_path)
else:
    dataset = MeshDataset.load(dataset_path)

Mesh s_chair_office.glb has 6894 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_chair_office_01.glb has 25192 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_desk.glb has 7240 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_door_double.glb has 4424 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_form_torso_dress_feminine.glb has 21754 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_sgcontroller.glb has 8848 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_skate_park.glb has 8598 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_table_coffee.glb has 4304 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_table_coffee_variation.glb has 8140 faces which is more than the allowed 4096 faces. Rejecting.
Mesh s_vehicle_bicycle_bmx.glb has 39972 faces which is more than the allowed 4096 faces. Rejecting.
Mesh sk_baby_01.glb has 13408 faces which is 

### Inspect

In [29]:
seen_texts = []
mesh_list = []  # List to store individual meshes
translation_distance = 1.0  # Distance to translate vertices, set to 1 unit as required

# Iterate over each item in the dataset
for r, item in enumerate(dataset.data):
    # Get vertices and faces
    vertices = np.array(item['vertices'].cpu())
    faces = np.array(item['faces'].cpu())
    texts = item['texts']

    if texts in seen_texts:
      continue
    seen_texts.append(texts)

    # Translate the vertices copy
    translation_vector = np.array([len(seen_texts) * translation_distance, 0, 0])  # Translation along the x-axis
    vertices[:, :3] += translation_vector  # Apply translation only to x, y, z

    # Create a new mesh object with the translated vertices and original faces
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    # Append the new mesh to our list of meshes
    mesh_list.append(mesh)

    print(f"Iteration {r} complete. Processed texts {texts} with {len(vertices)} vertices and {len(faces)} faces.")

# After iterating over all items, print the number of processed meshes
print(f"Total number of processed meshes: {len(mesh_list)}")


Iteration 0 complete. Processed texts Mire Clothing with 2179 vertices and 3884 faces.
Iteration 1 complete. Processed texts S Bed Full with 62 vertices and 48 faces.
Iteration 2 complete. Processed texts S Bed King with 403 vertices and 480 faces.
Iteration 3 complete. Processed texts S Bed Twin with 403 vertices and 480 faces.
Iteration 4 complete. Processed texts S Bone with 56 vertices and 24 faces.
Iteration 5 complete. Processed texts S Box with 24 vertices and 12 faces.
Iteration 6 complete. Processed texts S Cabinet Bookshelf with 690 vertices and 512 faces.
Iteration 7 complete. Processed texts S Cabinet Dresser 03 with 552 vertices and 256 faces.
Iteration 8 complete. Processed texts S Cabinet Dresser 05 with 744 vertices and 344 faces.
Iteration 9 complete. Processed texts S Chair Bar with 385 vertices and 440 faces.
Iteration 10 complete. Processed texts S Chair Box with 1848 vertices and 912 faces.
Iteration 11 complete. Processed texts S Chair Modern with 904 vertices and

### Train!

In [30]:
autoencoder = MeshAutoencoder().to("cuda")

**Have at least 400-2000 items in the dataset, use this to multiply the dataset**  

In [31]:
import random


random.seed(42)
random.shuffle(dataset.data)

initial_length = len(dataset.data)
target_length = 2000
print(initial_length)
if initial_length > 0:
    replication_factor = max(1, (target_length - 1) // initial_length + 1)
    dataset.data *= replication_factor
    dataset.data = dataset.data[:target_length]

print(len(dataset.data))

44
2000


**Train to about 0.3 loss if you are using a small dataset**

In [32]:
autoencoder_trainer = MeshAutoencoderTrainer(model = autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                            batch_size=8,
                                            grad_accum_every=2,
                                            learning_rate = 4e-3)
if is_train_autoencoder:
  if not is_train_autoencoder_disable_iteration:
    autoencoder_trainer.load(f'{working_dir}/mesh-encoder_{project_name}.pt')
  loss = autoencoder_trainer.train(380,stop_at_loss = 0.28, diplay_graph= True)
  autoencoder_trainer.save(f'{working_dir}/mesh-encoder_{project_name}.pt')
else:
  autoencoder_trainer.load(f'{working_dir}/mesh-encoder_{project_name}.pt')
  autencoder = autoencoder_trainer.model
  for param in autoencoder.parameters():
      param.requires_grad = True
  import gc

Epoch 1/380:   0%|                                                                                                                                                     | 0/250 [00:00<?, ?it/s]

Epoch 1/380: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:51<00:00,  4.83it/s, commit_loss=0.538, loss=0.921, recon_loss=0.867]


Epoch 1 average loss: 1.0335369660854339 recon loss: 0.9772: commit_loss 0.5633


Epoch 2/380: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:47<00:00,  5.23it/s, commit_loss=0.543, loss=0.763, recon_loss=0.709]


Epoch 2 average loss: 0.8711508121490479 recon loss: 0.8294: commit_loss 0.4174


...

Epoch 108/380: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:46<00:00,  5.32it/s, commit_loss=-0.541, loss=0.283, recon_loss=0.337]


Epoch 108 average loss: 0.2809475254416466 recon loss: 0.3367: commit_loss -0.5572          avg loss speed: -0.00037859563032788257


Epoch 109/380: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:47<00:00,  5.24it/s, commit_loss=-1.04, loss=0.233, recon_loss=0.337]


Epoch 109 average loss: 0.28104747307300565 recon loss: 0.3367: commit_loss -0.5561          avg loss speed: -0.0004682168563206468


Epoch 110/380:  74%|████████████████████████████████████████████████████████████████████████████████████                             | 186/250 [00:35<00:09,  6.48it/s, commit_loss=-0.586, loss=0.278, recon_loss=0.337]

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

max_length =  max(len(d["faces"]) for d in dataset if "faces" in d)
max_seq = max_length * 6
print("Highest face count:" , max_length)
print("Max token sequence:" , max_seq)

transformer = MeshTransformer(
    autoencoder,
    dim = 512,
    coarse_pre_gateloop_depth = 6, # Better performance using more gateloop layers
    fine_pre_gateloop_depth= 4,
    #attn_depth = 24, # GPT-2 medium have 24 layer depth, change if needed
    max_seq_len = max_seq,
    condition_on_text = True,
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", ## Change or remove this line if you are using:  https://github.com/MarcusLoppe/classifier-free-guidance-pytorch
    text_condition_cond_drop_prob = 0.0
)

total_params = sum(p.numel() for p in transformer.decoder.parameters())
total_params = f"{total_params / 1000000:.1f}M"
print(f"Decoder total parameters: {total_params}")
total_params = sum(p.numel() for p in transformer.parameters())
total_params = f"{total_params / 1000000:.1f}M"
print(f"Total parameters: {total_params}")

## **Required!**, embed the text and run generate_codes to save 4-96 GB VRAM (dependant on dataset) ##

**If you don't;** <br>
During each during each training step the autoencoder will generate the codes and the text encoder will embed the text.
<br>
After these fields are generate: **they will be deleted and next time it generates the code again:**<br>

This is due to the dataloaders nature, it writes this information to a temporary COPY of the dataset


In [ ]:
labels = set(item["texts"] for item in dataset.data)
print(labels)
dataset.embed_texts(transformer)
dataset.generate_codes(autoencoder)
print(dataset.data[0].keys())

*Load previous saved model if you had to restart session*

**Train to about 0.0001 loss (or less) if you are using a small dataset**

In [ ]:
if is_train_mesh_transformer:
  trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=4,num_train_steps=100, dataset = dataset,
                                 learning_rate = 5e-4, batch_size=1)
  loss = trainer.train(200, stop_at_loss = 0.00001)
else:
  trainer = MeshTransformerTrainer(model = transformer, warmup_steps = 10,grad_accum_every=1,num_train_steps=100, dataset = dataset, learning_rate = 1e-1, batch_size=1)
  trainer.load(f'{working_dir}/mesh-transformer_{project_name}.pt')
  transformer = trainer.model

**Load the latest model**

In [ ]:
autoencoder_trainer = MeshAutoencoderTrainer(model = autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100, batch_size=1,  grad_accum_every=1, learning_rate = 1e-4)
autoencoder_trainer.load(f'{working_dir}/mesh-encoder_{project_name}.pt')
autencoder = autoencoder_trainer.model
for param in autoencoder.parameters():
    param.requires_grad = True
import gc
torch.cuda.empty_cache()
gc.collect()

max_length =  max(len(d["faces"]) for d in dataset if "faces" in d)
max_seq = max_length * 6
print("Highest face count:" , max_length)
print("Max token sequence:" , max_seq)

transformer = MeshTransformer(
    autoencoder,
    dim = 512,
    coarse_pre_gateloop_depth = 6, # Better performance using more gateloop layers
    fine_pre_gateloop_depth= 4,
    # attn_depth = 24, # GPT-2 medium have 24 layer depth, change if needed
    max_seq_len = max_seq,
    condition_on_text = True,
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", ## Change or remove this line if you are using:  https://github.com/MarcusLoppe/classifier-free-guidance-pytorch
    text_condition_cond_drop_prob = 0.0
).to("cuda")

trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=1,num_train_steps=100, dataset = dataset, learning_rate = 1e-1, batch_size=2)
trainer.load(f'{working_dir}/mesh-transformer_{project_name}.pt')
transformer = trainer.model


## Generate and view mesh

In [ ]:
trainer.save(f'{working_dir}/mesh-transformer_{project_name}.pt')

In [ ]:
def combind_mesh(path, mesh):
    all_vertices = []
    all_faces = []
    vertex_offset = 0
    translation_distance = 0.5

    for r, faces_coordinates in enumerate(mesh):
        numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)

        for vertex in numpy_data:
            all_vertices.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

        for i in range(1, len(numpy_data), 3):
            all_faces.append(f"f {i + vertex_offset} {i + 1 + vertex_offset} {i + 2 + vertex_offset}\n")

        vertex_offset += len(numpy_data)

    obj_file_content = "".join(all_vertices) + "".join(all_faces)

    with open(path , "w") as file:
        file.write(obj_file_content)

def combind_mesh_with_rows(path, meshes):
    all_vertices = []
    all_faces = []
    vertex_offset = 0
    translation_distance = 0.5

    for row, mesh in enumerate(meshes):
        for r, faces_coordinates in enumerate(mesh):
            numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)
            numpy_data[:, 0] += translation_distance * (r / 0.2 - 1)
            numpy_data[:, 2] += translation_distance * (row / 0.2 - 1)

            for vertex in numpy_data:
                all_vertices.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

            for i in range(1, len(numpy_data), 3):
                all_faces.append(f"f {i + vertex_offset} {i + 1 + vertex_offset} {i + 2 + vertex_offset}\n")

            vertex_offset += len(numpy_data)

        obj_file_content = "".join(all_vertices) + "".join(all_faces)

    with open(path , "w") as file:
        file.write(obj_file_content)


def write_mesh_output(path, coords):
    numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)
    obj_file_content = ""

    for vertex in numpy_data:
        obj_file_content += f"v {vertex[0]} {vertex[1]} {vertex[2]}\n"

    for i in range(1, len(numpy_data), 3):
        obj_file_content += f"f {i} {i + 1} {i + 2}\n"

    with open(path, "w") as file:
        file.write(obj_file_content)


**Using only text**

In [ ]:

from pathlib import Path

folder = working_dir / 'renders'
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)

text_coords = []
for text in labels:
    print(f"Generating {text}")
    faces_coordinates = transformer.generate(texts = [text],  temperature = 0.0)
    text_coords.append(faces_coordinates)

    write_mesh_output(f'{folder}/3d_output_{text}.obj', faces_coordinates)


combind_mesh(f'{folder}/3d_models_all.obj', text_coords)

# **Preview 3d Models**

In [ ]:
!pip install ipyvolume trimesh pythreejs
import trimesh

folder = working_dir / 'renders'

mesh = trimesh.load(f'{folder}/3d_models_all.obj')
import ipyvolume as ipv

vertices = mesh.vertices
faces = mesh.faces

fig = ipv.figure()
ipv.plot_trisurf(vertices[:,0], vertices[:,1], vertices[:,2], triangles=faces)
ipv.show()

**Text + prompt of tokens**

Grab fresh copy of dataset

In [ ]:
dataset = MeshDataset.load(dataset_path)
dataset.generate_codes(autoencoder)

**Prompt with 10% of codes/tokens**

In [ ]:
from pathlib import Path
token_length_procent = 0.10
codes = []
texts = []
for label in labels:
    for item in dataset.data:
        if item['texts'] == label:
            num_tokens = int(item["codes"].shape[0] * token_length_procent)

            texts.append(item['texts'])
            codes.append(item["codes"].flatten()[:num_tokens].unsqueeze(0))
            break

folder = working_dir / f'renders/text+codes'
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)

coords = []



for text, prompt in zip(texts, codes):
    print(f"Generating {text} with {prompt.shape[1]} tokens")
    faces_coordinates = transformer.generate(texts = [text],  prompt = prompt, temperature = 0)
    coords.append(faces_coordinates)

    obj_file_path = f'{folder}/{text}_{prompt.shape[1]}_tokens.obj'
    write_mesh_output(obj_file_path, faces_coordinates)

    print(obj_file_path)


combind_mesh(f'{folder}/text+prompt_all.obj', coords)

if text_coords is not None:
    combind_mesh_with_rows(f'{folder}/both_verisons.obj', [text_coords , coords])

**Prompt with 0% to 80% of tokens**

In [ ]:
import json
import numpy as np
import trimesh
from pathlib import Path

def convert_to_obj(vertices, faces, output_file_path):
    scene = trimesh.Scene()
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    scene.add_geometry(mesh)

    with open(output_file_path, "w") as f:
        f.write(scene.export(file_type="obj"))

def encode_to_pua(codes):
    flat_codes = [item for sublist in codes for subsublist in sublist for item in subsublist]
    return "".join(chr(code + 0xF0000) for code in flat_codes)

jsonl_lines = []

for token_length_procent in np.arange(0, 0.8, 0.1):
    codes = []
    texts = []
    for label in labels:
        for item in dataset.data:
            if item['texts'] == label:
                num_tokens = int(item["codes"].shape[0] * token_length_procent)

                texts.append(item['texts'])
                codes.append(item["codes"].flatten()[:num_tokens].unsqueeze(0))
                break

    coords = []
    for text, code in zip(texts, codes):
        print(f"Generating {text} with {code.shape[1]} tokens")
        faces_coordinates = transformer.generate(texts=[text], prompt=code, temperature=0)
        coords.append(faces_coordinates)

        # Process mesh data inlined here
        continuous_coors_list = [np_array.tolist() for np_array in faces_coordinates]
        flat_list = [item for sublist in continuous_coors_list for item in sublist]
        vertices = [vertex for sublist in flat_list for vertex in sublist]
        faces = [[i, i + 1, i + 2] for i in range(0, len(vertices), 3)]

        obj_filename = f'{text}_{code.shape[1]}_tokens.obj'
        obj_file_path = folder / obj_filename
        convert_to_obj(vertices, faces, obj_file_path)

        encoded_codes = encode_to_pua(code.cpu().tolist())

        with open(obj_file_path, "r") as file:
            obj_contents = file.read()

        # Append line to JSONL structure
        jsonl_line = [
            {"role": "system", "content": "This assistant can understand 3D models using the meshgpt-pytorch Unicode plane 15 16384 item codebook for triangles and the .obj 3d format."},
            {"role": "user", "content": encoded_codes},
            {"role": "assistant", "content": obj_contents}
        ]
        jsonl_lines.append(jsonl_line)

        print(obj_file_path)

    mesh_rows.append(coords)
    combind_mesh(f'{folder}/text+prompt_all_{token_length_procent}.obj', coords)

combind_mesh_with_rows(f'{folder}/all.obj', mesh_rows)

with open("chatml.jsonl", "w", encoding="utf-8") as f:
    for item in jsonl_lines:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")